In [ ]:
import numpy as np
import ast
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 텍스트 파일 열기
with open('/content/output_file(m).txt', 'r', encoding='utf-8') as file:
    # 파일의 내용을 읽어서 문자열로 가져옴
    data_string = file.read().strip()

# 문자열을 리스트로 변환
data_list = ast.literal_eval(data_string)

X = []
y = []
for actor in data_list : # 모든 배우 순회
    for i in range(1,len(actor)) : # 배우 내 이미지 수 만큼
        num = []
        try :
            for key, landmark in actor[i].items(): # 벡터값만 추출
                for tup in landmark:
                    num.extend(tup)
            X.append(num)
            y.append(actor[0])
        except :
            continue

X = np.array(X)
y = np.array(y)

# 레이블 숫자로 변환
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 특성 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 정의
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X.shape[1],)))
model.add(Dropout(0.1))  # 드롭아웃 추가
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))  # 드롭아웃 추가
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))  # 드롭아웃 추가
model.add(Dense(len(np.unique(y)), activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test))

# history 객체에는 각 epoch에 대한 훈련 및 검증 정확도가 저장되어 있습니다.
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Accuracy:", history.history['val_accuracy'][-1])

# 학습 과정의 정확도 시각화
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# 학습 과정의 손실 시각화
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()